In [2]:
import pandas as pd
import numpy as np

In [9]:
# categorical_df = pd.read_csv('../dataset/5_categorical_data.csv')
# numerical_df = pd.read_csv('../dataset/5_numerical_data.csv')

categorical_df = pd.read_csv('../dataset/theoretical_categorical_data.csv')
numerical_df = pd.read_csv('../dataset/theoretical_numerical_data.csv')


categorical_df.head()

/var/folders/xk/fhgbj8qj60n54b05mpfnck0w0000gn/T/ipykernel_99967/1760696186.py:4: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  categorical_df = pd.read_csv('../dataset/theoretical_categorical_data.csv')


,formula_pretty,symmetry,structure,decomposes_to,xas,is_gap_direct,is_metal,bandstructure,dos,ordering,bulk_modulus
0,Si,"{'crystal_system': 'Triclinic', 'symbol': 'P1'...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-149', 'formula': 'Si2', '...","[{'edge': 'K', 'absorbing_element': 'Si', 'spe...",False,True,{'setyawan_curtarolo': {'task_id': 'mp-1049050...,"{'total': {'1': {'task_id': 'mp-1049056', 'ban...",NM,NaN
1,Ti,"{'crystal_system': 'Triclinic', 'symbol': 'P1'...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-72', 'formula': 'Ti3', 'a...",NaN,False,True,NaN,NaN,NM,NaN
2,C,"{'crystal_system': 'Triclinic', 'symbol': 'P1'...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-2516584', 'formula': 'C4'...",NaN,False,False,NaN,NaN,FM,NaN
3,Zn,"{'crystal_system': 'Triclinic', 'symbol': 'P1'...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-79', 'formula': 'Zn2', 'a...",NaN,False,True,NaN,NaN,NM,NaN
4,Si,"{'crystal_system': 'Triclinic', 'symbol': 'P1'...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-149', 'formula': 'Si2', '...",NaN,False,False,NaN,NaN,NM,NaN


In [10]:
#begin work on symmetry attribute
import ast

crystal_sytems_df = []

for row in categorical_df.itertuples():
   
    symmetry_dict = ast.literal_eval(row.symmetry) 
    crystal_sytems_df.append(symmetry_dict['crystal_system'])

crystal_sytems_df = pd.get_dummies(
    crystal_sytems_df, columns=['crystal_system'], 
                                    drop_first=True, prefix='crystal_sys')

crystal_sytems_df = crystal_sytems_df.astype(int)
categorical_df = categorical_df.drop('symmetry', axis=1)

numerical_df = pd.concat([numerical_df, crystal_sytems_df], axis=1)
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105589 entries, 0 to 105588
Data columns (total 41 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   nsites                                        105589 non-null  int64  
 1   nelements                                     105589 non-null  int64  
 2   volume                                        105589 non-null  float64
 3   density                                       105589 non-null  float64
 4   density_atomic                                105589 non-null  float64
 5   energy_per_atom                               103475 non-null  float64
 6   formation_energy_per_atom                     103475 non-null  float64
 7   energy_above_hull                             103475 non-null  float64
 8   is_stable                                     105589 non-null  int64  
 9   equilibrium_reaction_energy_per_atom          10

In [11]:
import ast

lattice_matrix_df = []

for row in categorical_df.itertuples(index=False):
    try:
        structure_dict = ast.literal_eval(row.structure)
        
        if 'lattice' in structure_dict and 'matrix' in structure_dict['lattice']:
            lattice_matrix = structure_dict['lattice']['matrix']
            lattice_matrix_df.append(lattice_matrix)

    except (ValueError, SyntaxError) as e:
        lattice_matrix_df.append("none")
lattice_matrix_df = pd.DataFrame(lattice_matrix_df)

print(lattice_matrix_df.head())

                                       0  \
0                   [4.934894, 0.0, 0.0]   
1   [12.49654657, 0.17686483, 0.2294622]   
2       [10.247851, -0.106168, 0.008782]   
3  [11.8521467, 0.55559273, -0.00317713]   
4  [11.61272298, 0.89401036, 1.03378931]   

                                        1  \
0              [-2.219485, 4.913732, 0.0]   
1  [0.17823107, 11.45138636, -0.31319981]   
2        [-0.098855, 10.894242, 0.432224]   
3   [0.55644679, 11.72989848, 0.31759797]   
4   [0.79773741, 13.36727246, -0.8604892]   

                                        2  
0         [-2.418904, -1.422197, 6.56246]  
1  [0.22747837, -0.31303906, 12.33339993]  
2         [0.005544, 0.456147, 10.113102]  
3  [-0.01663057, 0.31487646, 11.71135177]  
4  [1.03530586, -0.94340276, 12.59653651]  


In [13]:
#Convert matrices to a single value using Frobeius norm:

def frobenius_norm(matrix_str):
    try:
        matrix = np.array(ast.literal_eval(matrix_str))
        return np.linalg.norm(matrix, 'fro')
    except (ValueError, SyntaxError):
        return None
    
norms_df = []

for row in lattice_matrix_df.itertuples(index=False):
    matrix=row[0]
    norm = frobenius_norm(matrix)
    norms_df.append(norm)

norms_df = pd.DataFrame(norms_df)
print(norms_df)

           0
0       None
1       None
2       None
3       None
4       None
...      ...
105584  None
105585  None
105586  None
105587  None
105588  None

[105589 rows x 1 columns]


In [14]:
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105589 entries, 0 to 105588
Data columns (total 41 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   nsites                                        105589 non-null  int64  
 1   nelements                                     105589 non-null  int64  
 2   volume                                        105589 non-null  float64
 3   density                                       105589 non-null  float64
 4   density_atomic                                105589 non-null  float64
 5   energy_per_atom                               103475 non-null  float64
 6   formation_energy_per_atom                     103475 non-null  float64
 7   energy_above_hull                             103475 non-null  float64
 8   is_stable                                     105589 non-null  int64  
 9   equilibrium_reaction_energy_per_atom          10

In [ ]:
categorical_df.info()

#FOR NEXT TIME. WE NEED TO DROP ALL NULL FOR 4 AND 5 BC THESE ARE IMPORTANT ATTRIBUTES
#1 HOT ENCODE THEM TO INT 1,0s
#ADD TO NUMERICAL
#TRAIN AND TEST A MODEL

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105589 entries, 0 to 105588
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   formula_pretty  105587 non-null  object
 1   structure       105589 non-null  object
 2   decomposes_to   92800 non-null   object
 3   xas             23692 non-null   object
 4   is_gap_direct   105581 non-null  object
 5   is_metal        105581 non-null  object
 6   bandstructure   36421 non-null   object
 7   dos             49763 non-null   object
 8   ordering        104799 non-null  object
 9   bulk_modulus    4830 non-null    object
dtypes: object(10)
memory usage: 8.1+ MB
